In [1]:
!pip3 install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 41.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 43.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 52.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 22.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 42.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.37.0-py3-none-any.whl size=25320578 sha256=e222

In [2]:
import os
import string
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import tensorflow as tf
import ktrain
from ktrain import text
from spacy import displacy
from sklearn.model_selection import train_test_split

nlp = spacy.load('en_core_web_lg')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
dir(text)

['AnswerExtractor',
 'EnglishTranslator',
 'SimpleQA',
 'TextExtractor',
 'Transformer',
 'TransformerEmbedding',
 'TransformerSummarizer',
 'Translator',
 'ZeroShotClassifier',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'data',
 'eda',
 'entities_from_array',
 'entities_from_conll2003',
 'entities_from_df',
 'entities_from_gmb',
 'entities_from_txt',
 'extract_filenames',
 'filter_by_id',
 'get_topic_model',
 'learner',
 'load_text_files',
 'load_topic_model',
 'models',
 'ner',
 'pickle',
 'predictor',
 'preprocessor',
 'print_sequence_taggers',
 'print_text_classifiers',
 'print_text_regression_models',
 'qa',
 'seqlen_stats',
 'sequence_tagger',
 'shallownlp',
 'summarization',
 'text_classifier',
 'text_regression_model',
 'textextractor',
 'texts_from_array',
 'texts_from_csv',
 'texts_from_df',
 'texts_from_folder',
 'textutils',
 'translation',
 'zsl']

## Read Data

In [4]:
root_folder = r'/kaggle/input/20news-18828/20news-18828'
sub_folders = os.listdir(root_folder)
len(sub_folders)

20

In [5]:
sub_folders

['soc.religion.christian',
 'talk.politics.misc',
 'rec.motorcycles',
 'talk.politics.mideast',
 'sci.electronics',
 'sci.space',
 'rec.sport.baseball',
 'misc.forsale',
 'talk.politics.guns',
 'sci.med',
 'comp.sys.mac.hardware',
 'rec.sport.hockey',
 'comp.sys.ibm.pc.hardware',
 'talk.religion.misc',
 'sci.crypt',
 'comp.windows.x',
 'comp.os.ms-windows.misc',
 'rec.autos',
 'comp.graphics',
 'alt.atheism']

In [6]:
def read_data(root_folder, sub_folders):

    data, labels = [], []
    
    for folder in sub_folders:
        path = os.path.join(root_folder, folder)
        samples = os.listdir(path)

        for sample in samples:
            sample_path = os.path.join(path, sample)
            text = open(sample_path, encoding='iso-8859-1')   # 'latin-1')
            data.append(text.read())
            labels.append(folder)
    
    return data, labels

In [7]:
data, labels = read_data(root_folder, sub_folders)

print(len(data))
print(len(labels))

18828
18828


In [8]:
text.texts_from_array?

Signature:
text.texts_from_array(
    x_train,
    y_train,
    x_test=None,
    y_test=None,
    class_names=[],
    max_features=20000,
    maxlen=400,
    val_pct=0.1,
    ngram_range=1,
    preprocess_mode='standard',
    lang=None,
    random_state=None,
    verbose=1,
)
Docstring:
```
Loads and preprocesses text data from arrays.
texts_from_array can handle data for both text classification
and text regression.  If class_names is empty, a regression task is assumed.
Args:
    x_train(list): list of training texts
    y_train(list): labels in one of the following forms:
                   1. list of integers representing classes (class_names is required)
                   2. list of strings representing classes (class_names is not needed and ignored.)
                   3. a one or multi hot encoded array representing classes (class_names is required)
                   4. numerical values for text regresssion (class_names should be left empty)
    x_test(list): list of training 

In [9]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=data,
                                                                      y_train=labels,
                                                                      val_pct=0.2,
                                                                      maxlen=512,
                                                                      random_state=None,
                                                                      preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [10]:
x_train

[array([[ 101, 2013, 1024, ..., 4171, 1011,  102],
        [ 101, 2013, 1024, ..., 1035, 1010,  102],
        [ 101, 2013, 1024, ...,    0,    0,    0],
        ...,
        [ 101, 2013, 1024, ...,    0,    0,    0],
        [ 101, 2013, 1024, ...,    0,    0,    0],
        [ 101, 2013, 1024, ...,    0,    0,    0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [11]:
print(len(x_train))
print(len(x_train[0]))
print(len(x_train[1]))

2
15062
15062


In [12]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [13]:
print(len(y_train))
print(len(y_train[0]))

15062
20


In [14]:
preproc

In [15]:
dir(preproc)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_transform_y',
 'c',
 'check_trained',
 'get_classes',
 'get_preprocessor',
 'get_tokenizer',
 'is_chinese',
 'is_nospace_lang',
 'lang',
 'max_features',
 'maxlen',
 'migrate_classes',
 'multilabel',
 'ngram_range',
 'preprocess',
 'preprocess_test',
 'preprocess_train',
 'preprocess_train_called',
 'print_seqlen_stats',
 'process_chinese',
 'seqlen_stats',
 'set_classes',
 'set_multilabel',
 'tok',
 'tok_dct',
 'undo',
 'ytransform']

## Model

In [16]:
model = text.text_classifier(name='bert',
                             train_data=(x_train, y_train),
                             preproc=preproc)

Is Multi-Label? False
maxlen is 512


/opt/conda/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


In [17]:
learner = ktrain.get_learner(model=model,
                             train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=6)

## Training

In [18]:
learner.fit_onecycle(lr=2e-5,
                     epochs=1)



begin training using onecycle policy with max lr of 2e-05...
2511/2511 [==============================] - 1387s 543ms/step - loss: 0.9729 - accuracy: 0.7080 - val_loss: 0.3566 - val_accuracy: 0.8853


In [19]:
learner.evaluate((x_train, y_train))

471/471 [==============================] - 346s 726ms/step
              precision    recall  f1-score   support

           0       0.84      0.81      0.83       631
           1       0.89      0.90      0.89       777
           2       0.88      0.89      0.89       789
           3       0.82      0.84      0.83       804
           4       0.93      0.87      0.90       765
           5       0.95      0.93      0.94       786
           6       0.91      0.95      0.93       787
           7       0.96      0.96      0.96       789
           8       0.97      0.95      0.96       771
           9       0.99      0.98      0.98       821
          10       0.99      0.99      0.99       782
          11       0.98      0.97      0.98       790
          12       0.92      0.93      0.92       792
          13       0.99      0.98      0.98       775
          14       0.99      0.97      0.98       799
          15       0.86      0.98      0.92       795
          16       0.9

array([[514,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,
          0,   2,  19,   2,   4,   3,  86],
       [  0, 702,  25,  14,   6,  15,   5,   0,   0,   2,   0,   3,   1,
          1,   1,   1,   0,   1,   0,   0],
       [  0,  24, 699,  39,   3,  15,   3,   3,   0,   0,   0,   1,   1,
          1,   0,   0,   0,   0,   0,   0],
       [  0,  13,  42, 675,  26,   4,  14,   1,   2,   0,   0,   2,  23,
          1,   0,   1,   0,   0,   0,   0],
       [  0,   6,   1,  61, 669,   0,  18,   0,   0,   0,   0,   0,   9,
          1,   0,   0,   0,   0,   0,   0],
       [  0,  25,  19,   4,   2, 728,   4,   0,   0,   0,   0,   0,   0,
          0,   2,   0,   0,   1,   0,   1],
       [  0,   0,   2,  10,   2,   0, 750,   7,   6,   0,   0,   0,   9,
          1,   0,   0,   0,   0,   0,   0],
       [  0,   1,   1,   0,   0,   1,  12, 759,   9,   0,   0,   0,   5,
          0,   0,   0,   0,   1,   0,   0],
       [  3,   0,   0,   0,   0,   0,   2,  16, 736,   0,   1,  

In [20]:
learner.evaluate((x_test, y_test))

118/118 [==============================] - 86s 725ms/step
              precision    recall  f1-score   support

           0       0.72      0.72      0.72       168
           1       0.85      0.85      0.85       196
           2       0.85      0.87      0.86       196
           3       0.72      0.79      0.75       178
           4       0.92      0.85      0.88       196
           5       0.95      0.90      0.92       194
           6       0.88      0.91      0.89       185
           7       0.95      0.95      0.95       201
           8       0.95      0.92      0.94       223
           9       0.98      0.96      0.97       173
          10       0.99      0.97      0.98       217
          11       0.95      0.95      0.95       201
          12       0.88      0.87      0.88       189
          13       0.97      0.97      0.97       215
          14       0.94      0.95      0.95       188
          15       0.83      0.93      0.88       202
          16       0.87

array([[121,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          2,   3,   6,   1,   3,   3,  29],
       [  1, 166,   9,   5,   4,   3,   1,   0,   0,   0,   0,   3,   2,
          0,   0,   1,   0,   1,   0,   0],
       [  0,   4, 171,  17,   1,   1,   2,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   1,  13, 140,   4,   3,   8,   0,   0,   0,   0,   0,   9,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   1,   1,  19, 166,   1,   3,   1,   0,   0,   0,   0,   3,
          0,   0,   0,   1,   0,   0,   0],
       [  0,  13,   4,   1,   0, 174,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   1,   0,   1],
       [  0,   0,   0,   3,   1,   0, 168,   2,   4,   0,   1,   2,   3,
          0,   0,   0,   0,   0,   1,   0],
       [  0,   0,   0,   0,   0,   0,   3, 190,   5,   0,   0,   0,   0,
          0,   2,   0,   1,   0,   0,   0],
       [  1,   0,   1,   0,   0,   1,   1,   7, 205,   1,   0,  

In [28]:
learner.fit_onecycle(lr=2e-5,
                     epochs=1)



begin training using onecycle policy with max lr of 2e-05...
2511/2511 [==============================] - 1360s 542ms/step - loss: 0.2895 - accuracy: 0.9106 - val_loss: 0.2720 - val_accuracy: 0.9185


In [29]:
learner.evaluate((x_train, y_train))

471/471 [==============================] - 342s 726ms/step
              precision    recall  f1-score   support

           0       0.95      0.93      0.94       631
           1       0.95      0.97      0.96       777
           2       0.93      0.95      0.94       789
           3       0.91      0.91      0.91       804
           4       0.97      0.95      0.96       765
           5       0.99      0.96      0.98       786
           6       0.95      0.98      0.96       787
           7       0.99      0.98      0.99       789
           8       0.99      0.98      0.99       771
           9       0.99      1.00      1.00       821
          10       1.00      0.99      1.00       782
          11       0.99      0.99      0.99       790
          12       0.97      0.96      0.97       792
          13       0.99      0.99      0.99       775
          14       1.00      0.99      0.99       799
          15       0.96      0.99      0.98       795
          16       0.9

array([[585,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   2,   0,   1,   0,  43],
       [  0, 754,   9,   6,   1,   2,   3,   0,   0,   0,   0,   0,   0,
          0,   0,   1,   0,   1,   0,   0],
       [  0,  12, 752,  19,   2,   1,   1,   0,   0,   0,   0,   0,   1,
          0,   0,   0,   0,   1,   0,   0],
       [  0,   7,  34, 732,  11,   0,  12,   0,   0,   0,   0,   1,   7,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   2,   1,  25, 723,   0,  10,   0,   0,   0,   0,   0,   4,
          0,   0,   0,   0,   0,   0,   0],
       [  0,  14,  12,   2,   0, 757,   0,   0,   0,   0,   0,   0,   0,
          0,   1,   0,   0,   0,   0,   0],
       [  0,   0,   0,   6,   5,   0, 769,   1,   4,   0,   0,   0,   1,
          1,   0,   0,   0,   0,   0,   0],
       [  0,   1,   0,   0,   2,   0,   6, 776,   2,   0,   0,   0,   1,
          0,   0,   1,   0,   0,   0,   0],
       [  1,   0,   0,   0,   1,   0,   2,   4, 756,   3,   0,  

In [30]:
learner.evaluate((x_test, y_test))

118/118 [==============================] - 86s 725ms/step
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       168
           1       0.86      0.88      0.87       196
           2       0.85      0.91      0.88       196
           3       0.79      0.84      0.82       178
           4       0.91      0.90      0.90       196
           5       0.98      0.92      0.95       194
           6       0.90      0.90      0.90       185
           7       0.97      0.94      0.95       201
           8       0.94      0.93      0.94       223
           9       0.95      0.98      0.97       173
          10       0.97      0.96      0.97       217
          11       0.97      0.97      0.97       201
          12       0.92      0.89      0.91       189
          13       0.97      0.98      0.97       215
          14       0.97      0.96      0.96       188
          15       0.93      0.93      0.93       202
          16       0.90

array([[146,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          1,   1,   4,   1,   2,   0,  13],
       [  1, 172,  10,   3,   4,   2,   1,   0,   0,   0,   0,   1,   1,
          0,   0,   0,   0,   1,   0,   0],
       [  0,   4, 179,  10,   1,   0,   1,   0,   0,   0,   0,   0,   0,
          0,   1,   0,   0,   0,   0,   0],
       [  0,   2,  12, 150,   5,   0,   6,   0,   0,   0,   0,   0,   3,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   2,   2,   8, 176,   1,   3,   1,   0,   0,   0,   0,   3,
          0,   0,   0,   0,   0,   0,   0],
       [  0,  10,   4,   0,   0, 179,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   1,   0,   0],
       [  0,   1,   1,   4,   1,   0, 167,   1,   5,   0,   1,   1,   2,
          0,   0,   1,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   1,   3, 188,   7,   0,   1,   0,   0,
          0,   0,   0,   0,   0,   1,   0],
       [  1,   0,   0,   1,   0,   0,   1,   4, 208,   3,   0,  

In [34]:
learner.fit_onecycle(lr=2e-5,
                     epochs=1)



begin training using onecycle policy with max lr of 2e-05...
2511/2511 [==============================] - 1313s 523ms/step - loss: 0.1671 - accuracy: 0.9503 - val_loss: 0.2512 - val_accuracy: 0.9318


In [35]:
learner.evaluate((x_train, y_train))

471/471 [==============================] - 342s 726ms/step
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       631
           1       0.98      0.98      0.98       777
           2       0.97      0.96      0.97       789
           3       0.96      0.96      0.96       804
           4       0.99      0.97      0.98       765
           5       0.99      0.99      0.99       786
           6       0.97      0.99      0.98       787
           7       0.99      0.99      0.99       789
           8       0.99      0.99      0.99       771
           9       1.00      1.00      1.00       821
          10       1.00      1.00      1.00       782
          11       1.00      0.99      0.99       790
          12       0.98      0.99      0.98       792
          13       1.00      1.00      1.00       775
          14       1.00      1.00      1.00       799
          15       0.99      0.99      0.99       795
          16       0.9

array([[615,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   1,   0,   1,   0,  14],
       [  0, 763,   6,   4,   0,   2,   2,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,  10, 760,  18,   1,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   9, 771,   7,   4,   5,   0,   0,   0,   0,   0,   8,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   2,   2,   9, 743,   0,   7,   1,   0,   0,   0,   0,   1,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   2,   7,   0,   0, 776,   0,   0,   0,   0,   0,   0,   0,
          0,   1,   0,   0,   0,   0,   0],
       [  0,   0,   0,   1,   2,   0, 782,   0,   0,   0,   0,   0,   1,
          1,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   1,   5, 779,   2,   0,   0,   0,   2,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   2,   3, 766,   0,   0,  

In [36]:
learner.evaluate((x_test, y_test))

118/118 [==============================] - 85s 725ms/step
              precision    recall  f1-score   support

           0       0.90      0.89      0.90       168
           1       0.91      0.89      0.90       196
           2       0.90      0.92      0.91       196
           3       0.82      0.84      0.83       178
           4       0.93      0.91      0.92       196
           5       0.94      0.96      0.95       194
           6       0.89      0.94      0.91       185
           7       0.95      0.95      0.95       201
           8       0.97      0.94      0.95       223
           9       0.97      0.97      0.97       173
          10       0.98      0.98      0.98       217
          11       0.97      0.97      0.97       201
          12       0.93      0.91      0.92       189
          13       0.97      0.98      0.97       215
          14       0.96      0.97      0.97       188
          15       0.97      0.92      0.94       202
          16       0.93

array([[150,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   2,   0,   1,   2,   1,  12],
       [  0, 175,   7,   2,   5,   3,   0,   0,   0,   0,   0,   1,   2,
          0,   0,   0,   0,   1,   0,   0],
       [  0,   2, 181,  11,   0,   0,   1,   0,   0,   0,   0,   0,   0,
          0,   1,   0,   0,   0,   0,   0],
       [  0,   3,   6, 149,   4,   5,   6,   1,   0,   0,   0,   0,   4,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   2,   8, 178,   2,   4,   1,   0,   0,   0,   0,   1,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   5,   3,   0,   0, 186,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   2,   1,   0, 173,   1,   1,   0,   1,   3,   2,
          0,   0,   1,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   4, 190,   5,   0,   0,   0,   0,
          0,   1,   0,   0,   0,   1,   0],
       [  1,   0,   1,   0,   0,   0,   2,   4, 210,   2,   0,  

## deployment
https://medium.com/analytics-vidhya/how-to-deploy-your-neural-network-model-using-ktrain-ae255b134c77

## save 

In [39]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('/kaggle/working/')

## Load

In [23]:
# import pickle
# from tensorflow.keras.models import load_model

# # loading preprocess and model file
# features = pickle.load(open('/kaggle/working/tf_model.preproc','rb'))
# new_model = load_model('/kaggle/working/tf_model.h5')

## download

In [24]:
!cd /kaggle/working/

In [25]:
!ls

__notebook_source__.ipynb  tf_model.h5	tf_model.preproc


In [26]:
from IPython.display import FileLink

FileLink(r'tf_model.h5')

/kaggle/working/tf_model.h5

In [27]:
FileLink(r'tf_model.preproc')

/kaggle/working/tf_model.preproc